<a href="https://colab.research.google.com/github/Mercy-Njambi/Data-Lemur-Interview-Questions/blob/main/SQL/Hard/Repeated_Payments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sometimes, payment transactions are repeated by accident; it could be due to user error, API failure or a retry error that causes a credit card to be charged twice.

Using the transactions table, identify any payments made at the same merchant with the same credit card for the same amount within 10 minutes of each other. Count such repeated payments.

Assumptions:

- The first transaction of such payments should not be counted as a repeated payment. This means, if there are two transactions performed by a merchant with the same credit card and for the same amount within 10 minutes, there will only be 1 repeated payment.

`transactions` Table:

|Column Name	|Type|
|:---|:---|
|transaction_id	|integer|
|merchant_id|	integer|
|credit_card_id|	integer|
|amount|	integer|
|transaction_timestamp|	datetime|

Example Output:

|payment_count|
|:---|
|1|


In [ ]:
query = """
with ordered_data as (
select
  merchant_id,
  credit_card_id,
  transaction_timestamp,
  extract(epoch from(lead(transaction_timestamp) over(partition by merchant_id, credit_card_id order by transaction_timestamp) - transaction_timestamp)) seconds_difference,
  amount,
  lead(amount) over(partition by merchant_id, credit_card_id order by transaction_timestamp) next_amount
from transactions
order by merchant_id, credit_card_id, transaction_timestamp
)
select
  count(*) as payment_count
from ordered_data
where seconds_difference is not null
  and amount=next_amount
  and seconds_difference <=600

"""